In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# CSAF Imports
import csaf.config as cconf
import csaf.system as csys

# create a csaf configuration out of toml
my_conf = cconf.SystemConfig.from_toml("/csaf-system/f16_simple_config.toml")

In [ ]:
# termination condition
def ground_collision_condition(cname, outs):
        """ground collision premature termnation condition"""
        return cname == "plant" and outs["states"][11] <= 0.0

# create pub/sub components out of the configuration
my_system = csys.System.from_config(my_conf)

# create an environment from the system, allowing us to act as the controller
my_env = csys.SystemEnv("autopilot", my_system, terminating_conditions=ground_collision_condition)

In [ ]:
# collect aircraft states
pstates = []

# send signal of zeros
ctrl_signal = [0.,0.,0.,0.]

# step through simulation and collect f16 states
do_sim = True
while do_sim:
    comp_input = my_env.step({"autopilot-states": ["Waiting"], "autopilot-fdas": ["Waiting"], "autopilot-outputs": ctrl_signal})
    if isinstance(comp_input, bool):
        do_sim = False
    else:
        pstates.append(comp_input['plant-states'])
pstates = np.array(pstates)

In [ ]:
# plot the results
plt.plot(pstates[:, 11], label='F16 Altitude')
plt.xlabel("Step Index [n]")
plt.ylabel("[ft]")
plt.legend()
plt.show()